In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Data/final_metadata.csv')

In [3]:
df.shape

(19951, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19951 entries, 0 to 19950
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 19951 non-null  int64  
 1   title              19951 non-null  object 
 2   genres             19951 non-null  object 
 3   original_language  19951 non-null  object 
 4   overview           19951 non-null  object 
 5   popularity         19951 non-null  float64
 6   release_date       19951 non-null  object 
 7   vote_average       19951 non-null  float64
 8   vote_count         19951 non-null  float64
 9   keywords           19951 non-null  object 
 10  year               19951 non-null  int64  
 11  cast               19951 non-null  object 
 12  director           19951 non-null  object 
 13  score              19951 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 2.1+ MB


In [5]:
df.drop(columns=['release_date','vote_average', 'vote_count'], inplace=True)

In [38]:
# wiki_plot = pd.read_csv('Data/wiki_movie_plots.csv')
# wiki_plot.columns = wiki_plot.columns.str.lower()
# wiki_plot.rename(columns={'release year': 'year'}, inplace=True)
# wiki_plot_subset = wiki_plot[['year', 'title','plot']]

In [39]:
# wiki_plot_subset.head()

In [40]:
# final_df = pd.merge(df, wiki_plot_subset, how='left', on=['title','year'])

In [41]:
# final_df['plot'][final_df['id'] == 862].values[0]

In [42]:
# final_df['plot'].isna().sum()

In [43]:
# #Only Plot
# from bs4 import BeautifulSoup
# import requests

# def get_plot(movie_title):
#     base_url = "https://en.wikipedia.org/wiki/"
#     movie_title_formatted = movie_title.replace(' ', '_')
#     url = base_url + movie_title_formatted

#     try:
#         response = requests.get(url)
#         if response.status_code != 200:
#             raise Exception(f"Page not found for {movie_title}")

#         soup = BeautifulSoup(response.content, 'html.parser')

#         plot = ""
#         for header in soup.find_all(['h2', 'h3']):
#             if 'Plot' in header.text:
#                 next_node = header.find_next_sibling()
#                 while next_node and next_node.name not in ['h2', 'h3']:
#                     if next_node.name == 'p':
#                         plot += next_node.text
#                     next_node = next_node.find_next_sibling()
#                 break
        
#         plot = plot.strip() if plot else None
#         return plot
#     except Exception as e:
#         return None

In [44]:
# # Apply the get_director_from_tmdb function for missing directors
# from tqdm import tqdm

# missing_plots = final_df['plot'].isna()

# tqdm.pandas(desc="Retrieving plots from Wikipedia")
# final_df.loc[missing_plots, 'plot'] = final_df.loc[missing_plots, 'title'].progress_apply(get_plot)

In [45]:
# final_df['plot'].isna().sum()

In [51]:
# pip install weaviate-client

In [6]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json

In [7]:
openai_key = os.environ.get("OPENAI_API_KEY", "OPENAI_API_KEY")

client = weaviate.Client(
    url="https://movie-recommendation-system-5aj2u8op.weaviate.network",
    auth_client_secret=weaviate.auth.AuthApiKey("Ud8IiTwtZ73oYlFMZAfPwL1j8JHcaFYvbZOM"),
    additional_headers={
        "X-OpenAI-Api-Key": openai_key  # Replace with your inference API key
    }
)

c:\Users\aryan\AppData\Local\Programs\Python\Python311\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [17]:
movie_class_schema = {
    "class": "Movies",
    "description": "A collection of movies till 2024.",
    "vectorizer": "text2vec-huggingface",
    "moduleConfig": {
        "text2vec-huggingface": {
            "vectorizeClassName": False,
            "model": "sentence-transformers/all-MiniLM-L6-v2",
            "options": {
            "waitForModel": True,
            "useGPU": True,
            "useCache": True
          }
        },
    },
    "vectorIndexConfig": {"distance" : "cosine"},
}

In [18]:
df.columns

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'keywords', 'year', 'cast', 'director', 'score'],
      dtype='object')

In [19]:
movie_class_schema["properties"] = [
        {
            "name": "movie_id",
            "dataType": ["number"],
            "description": "The id of the movie", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }        
        },
        {
            "name": "title",
            "dataType": ["text"],
            "description": "The name of the movie", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }   
        },
        {
            "name": "genres",
            "dataType": ["text"],
            "description": "Genres of the movie.",       
        },
        {
            "name": "original_language",
            "dataType": ["text"],
            "description": "The language of the movie.", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }   
        },
        {
            "name": "overview",
            "dataType": ["text"],
            "description": "Synopsis of the movie.",        
        },
        {
            "name": "popularity",
            "dataType": ["number"],
            "description": "The popularity of the movie.", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }   
        },
        {
            "name": "keywords",
            "dataType": ["text"],
            "description": "The keywords associated with the movie.",        
        },
        {
            "name": "year",
            "dataType": ["number"],
            "description": "The year the movie was released.", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }   
        },
        {
            "name": "cast",
            "dataType": ["text"],
            "description": "Top 3 billed cast members.", 
        },
        {
            "name": "director",
            "dataType": ["text"],
            "description": "The movie's director.", 
        },
        {
            "name": "score",
            "dataType": ["number"],
            "description": "The weighted score of the movie.", 
            "moduleConfig": {
                "text2vec-openai": {  
                    "skip" : True,
                    "vectorizePropertyName" : False
                }
            }   
        },
    ]

# Create class
client.schema.create_class(movie_class_schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'multiple vectorizers configured in property\'s "movie_id" moduleConfig: [text2vec-openai text2vec-huggingface]. class.vectorizer is set to "text2vec-huggingface"'}]}.

In [12]:
df.shape

(19951, 11)

In [11]:
df.head()

,id,title,genres,original_language,overview,popularity,keywords,year,cast,director,score
0,823464,Godzilla x Kong: The New Empire,Science Fiction Action Adventure,English,Following their explosive showdown Godzilla an...,10484.676,giantmonster sequel dinosaur kaiju fantasyworl...,2024,RebeccaHall BrianTyreeHenry DanStevens,Adam Wingard,7.057562
1,615656,Meg 2: The Trench,Action Science Fiction Horror,English,An exploratory dive into the deepest depths of...,8763.998,basedonnovelorbook sequel kaiju,2023,JasonStatham WuJing ShuyaSophiaCai,Ben Wheatley,6.892359
2,758323,The Pope's Exorcist,Horror Mystery Thriller,English,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,spain romeitaly vatican pope pig possession co...,2023,RussellCrowe DanielZovatto AlexEssoe,Julius Avery,6.927192
3,667538,Transformers: Rise of the Beasts,Action Adventure Science Fiction,English,When a new threat capable of destroying the en...,5409.104,peru alien endoftheworld basedoncartoon basedo...,2023,AnthonyRamos DominiqueFishback LunaLaurenVelez,Steven Caple Jr.,7.014331
4,693134,Dune: Part Two,Science Fiction Adventure,English,Follow the mythic journey of Paul Atreides as ...,4742.163,epic basedonnovelorbook fight sandstorm sand s...,2024,TimothéeChalamet Zendaya RebeccaFerguson,Denis Villeneuve,7.973783


In [16]:
# Configure batch process - for faster imports 
client.batch.configure(batch_size=1)

# Importing the data
for i in range(len(df)):
    item = df.iloc[i]

    movie_object = {
        'movie_id':float(item['id']),
        'title': str(item['title']).lower(),
        'genres': str(item['genres']).lower(),
        'original_language': str(item['original_language']).lower(),
        'overview': str(item['overview']).lower(),
        'popularity':float(item['popularity']),
        'keywords': str(item['keywords']),
        'year':float(item['year']),
        'cast': str(item['cast']).lower(),
        'director': str(item['director']).lower(),
        'score':float(item['score']),
    }

    client.batch.add_data_object(movie_object, "Movies")

{'error': [{'message': 'connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more in

KeyboardInterrupt: 

### Getting vectors from CountVectorizor

In [17]:

# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(final_df['soup'])

In [18]:
count_matrix.shape

(19951, 49706)

In [19]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [20]:
indices = pd.Series(final_df.index, index=final_df['title'])

In [21]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim = cosine_sim):

    if title not in indices:
        raise ValueError(f"Title '{title}' not found in indices")
    
    # Get the index of the movie that matches the title
    idx = indices[title]  

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))   

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) 

    # Get the scores of the most similar movies
    sim_scores = sim_scores[1:10 + 1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]  

    # Getting the weighted ratings and popularity of the movies
    movies = final_df.iloc[movie_indices][['title', 'year', 'score', 'original_language', 'popularity']]

    # Sort movies first by 'Score' in descending order, then by 'popularity' in descending order
    movies = movies.sort_values(by=['score', 'popularity'], ascending=[False, False])

    return movies

## Tests

In [22]:
get_recommendations('The Dark Knight Rises')

,title,year,score,original_language,popularity
4,The Dark Knight,2008,8.474344,English,126.226
260,Batman Begins,2005,7.666825,English,93.893
1601,Batman: Assault on Arkham,2014,7.001524,English,14.829
5820,The Last Days of American Crime,2020,6.585388,English,14.945
8981,NH10,2015,6.500675,Hindi,4.279
11716,The Awakener,2018,6.439598,Portuguese,10.999
11804,Defendor,2009,6.437744,English,9.129
13227,Kick-Ass 2,2013,6.403846,English,31.942
16169,The Courier,2019,6.306215,English,22.071
18499,The Punisher,1989,6.129711,English,19.590


In [23]:
get_recommendations('Hulk')

,title,year,score,original_language,popularity
1684,Batman Beyond: Return of the Joker,2000,6.981537,English,19.496
3748,Planet Hulk,2010,6.692933,English,17.734
6945,Hulk vs. Thor,2009,6.550188,English,12.939
9464,Hulk: Where Monsters Dwell,2016,6.490451,English,8.846
12322,Avengers Confidential: Black Widow & Punisher,2014,6.426286,Japanese,37.412
17686,The Incredible Hulk,2008,6.218239,English,59.440
19505,Steel,1997,5.867048,English,12.966
19589,Fantastic Four,2005,5.817209,English,35.959
19609,Superman Returns,2006,5.807654,English,26.787
19948,Batman & Robin,1997,4.544024,English,36.578


In [24]:
get_recommendations('Your Name')

,title,year,score,original_language,popularity
2243,The Anthem of the Heart,2015,6.863976,Japanese,25.317
2372,Children Who Chase Lost Voices,2011,6.843095,Japanese,24.701
4390,I've Always Liked You,2016,6.648975,Japanese,21.744
4422,The Moment You Fall in Love,2016,6.647458,Japanese,22.387
4854,Until Forever,2016,6.625323,English,22.492
6107,Orange: Future,2016,6.575597,Japanese,5.132
6883,Her Blue Sky,2019,6.551745,Japanese,20.975
7224,"Love, Chunibyo & Other Delusions! Rikka Version",2013,6.542290,Japanese,15.995
7471,The Lost Husband,2020,6.535928,English,8.044
10973,Midori,1992,6.456913,Japanese,19.336


In [25]:
get_recommendations('The Godfather')

,title,year,score,original_language,popularity
3,The Godfather Part II,1974,8.484184,English,62.665
69,Scarface,1983,8.069943,English,68.125
423,The Irishman,2019,7.495517,English,27.739
700,The Godfather Part III,1990,7.306118,English,67.934
1905,The Godfather Trilogy: 1901-1980,1992,6.927905,English,11.684
2558,The Rainmaker,1997,6.818571,English,15.972
3343,Shoot the Piano Player,1960,6.728088,French,9.386
3398,...And Justice for All,1979,6.721785,English,10.820
5282,Salvatore Giuliano,1962,6.605344,Italian,5.686
8803,The Gambler,1974,6.504228,English,8.158


In [26]:
get_recommendations('Suzume')

,title,year,score,original_language,popularity
2323,Mirai,2018,6.849920,Japanese,30.796
2372,Children Who Chase Lost Voices,2011,6.843095,Japanese,24.701
2679,One Piece: Strong World,2009,6.804882,Japanese,31.282
3057,Naruto Shippuden the Movie: The Will of Fire,2009,6.758979,Japanese,47.101
3126,Drifting Home,2022,6.751755,Japanese,126.618
4916,Inuyasha the Movie 4: Fire on the Mystic Island,2004,6.622518,Japanese,28.915
8344,Pokémon the Movie: White - Victini and Zekrom,2011,6.514447,Japanese,13.737
8473,Naruto the Movie: Legend of the Stone of Gelel,2005,6.511735,Japanese,34.361
8518,Pokémon the Movie: Black - Victini and Reshiram,2011,6.510666,Japanese,14.226
8687,Modest Heroes,2018,6.506988,Japanese,25.804
